In [1]:
## No elimine esta celda, ya que permite actualizar los archivos .py
## sin tener que reiniciar el Kernel de Jupyter.
%load_ext autoreload
%autoreload 2
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torchvision.datasets import ImageFolder
import numpy as np
import torch
from train import find_best_model, run_experiment
from transforms import Transforms
from utils import  set_seed, count_instances_per_class
import os
import warnings


warnings.filterwarnings("ignore", message = "Palette images with Transparency expressed in bytes should be converted to RGBA images")
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.empty_cache()

set_seed()


In [2]:
train_transforms = A.Compose(
    [A.Resize(256, 256), A.ToFloat(), ToTensorV2()]
)
validation_transforms = A.Compose(
    [A.Resize(256, 256), A.ToFloat(), ToTensorV2()]
)


train_data = ImageFolder(
    "house_plant_species/train",
    transform = Transforms(train_transforms),
)
validation_data = ImageFolder(
    "house_plant_species/validation",
    transform = Transforms(validation_transforms),
)



print(f"Elementos en Entrenamiento: {len(train_data)}")
print(f"Elementos en Validación: {len(validation_data)}")
print(f"There are {len(train_data.classes)} classes!")
#

# Llamar a las funciones
class_counts = count_instances_per_class(train_data)

# Calcular estadísticas
class_instance_counts = np.array(list(class_counts.values()))
mean_instances = np.mean(class_instance_counts)
std_instances = np.std(class_instance_counts)
min_instances = np.min(class_instance_counts)
max_instances = np.max(class_instance_counts)

# # Mostrar resultados
# print("\nInstancias por clase:\n")
# for class_name, count in class_counts.items():
#     print(f"Clase {class_name}: {count} imágenes\n===========================================")

print("\nEstadísticas de las instancias por clase:\n")
print(f"Promedio de instancias:\n============================== {mean_instances:.2f}")
print(f"Desviación estándar:\n============================== {std_instances:.2f}")
print(f"Cantidad mínima de instancias:\n============================== {min_instances}")
print(f"Cantidad máxima de instancias:\n============================== {max_instances}")

Elementos en Entrenamiento: 10835
Elementos en Validación: 2736
There are 47 classes!

Estadísticas de las instancias por clase:

Promedio de instancias:
============================== 230.53
Desviación estándar:
============================== 77.46
Cantidad mínima de instancias:
============================== 49
Cantidad máxima de instancias:
============================== 432


### Best architecture is **efficientnet_b5.sw_in12k** obtained with:

###### "color_jitter", "horizontal_flip", "brightness_contrast", "blur"
###### Trivial fc_layers => Lazylinear(512) => Linear(512, num_clases)
###### Resize to 256 x 256 images
###### ToFloat() and not Normalize

### **New Change**
####         for layer in [self.backbone.conv_head, self.backbone.bn2, self.backbone.global_pool]:
####            for param in layer.parameters():
####                param.requires_grad = True
####  416 x 416 input images resizing


In [ ]:
run_experiment(
    selected_transforms = ["center_crop", "color_jitter", "horizontal_flip", "brightness_contrast", "noise", "blur"],
    model_name = "efficientnet_b5.sw_in12k",
    base_name = "",
    extra_info = "bn_drp_unfreeze_416", 
    model_type = "efficientnet_b5.sw_in12k",
    num_classes = 47,
    training_params = {"learning_rate": 0.0003, "batch_size": 32, "num_epochs": 50},
    frozen = True,
    result_file = "results/efficient_net.json",
    normalize = False,
    use_float = True,
    resize_size = 416  # Aquí defines dinámicamente el tamaño de `resize`
)

Running experiment: efficientnet_b5.sw_in12k__bn_drp_unfreeze_416
Using device: cuda


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: joseflores1 (benjaminrromo-universidad-adolfo-ib-ez). Use `wandb login --relogin` to force relogin


  2%|▏         | 1/50 [02:41<2:11:38, 161.20s/it]

Epoch: 1/50 - Time: 161.02s - Train Loss: 1.3811, Train Accuracy: 0.7659 - Validation Loss: 0.4153, Validation Accuracy: 0.9291
Model checkpoint saved at epoch 1


  4%|▍         | 2/50 [05:22<2:08:53, 161.11s/it]

...

Epoch: 11/50 - Time: 165.08s - Train Loss: 0.0740, Train Accuracy: 0.9794 - Validation Loss: 0.2012, Validation Accuracy: 0.9448


 22%|██▏       | 11/50 [32:40<1:55:49, 178.20s/it]

Epoch: 12/50 - Time: 164.22s - Train Loss: 0.0573, Train Accuracy: 0.9843 - Validation Loss: 0.2093, Validation Accuracy: 0.9470
Early stopping triggered at epoch 6. Best Validation Loss: 0.1831


elapsed_time,▁▁▁▄▃▇▅█▇██▇
epoch,▁▂▂▃▄▄▅▅▆▇▇█
train_acc,▁▆▇▇████████
train_loss,█▃▂▂▁▁▁▁▁▁▁▁
val_acc,▁▅▅▇▆█▆▇█▅▅▆
val_loss,█▃▂▁▁▁▁▁▁▂▂▂
elapsed_time,164.22048
epoch,12
train_acc,0.98428
train_loss,0.05734
val_acc,0.947


Results to save: {'architecture': 'efficientnet_b5.sw_in12k__bn_drp_unfreeze_416', 'model_type': 'efficientnet_b5.sw_in12k', 'best_epoch': 6, 'best_val_loss': 0.1831230824369244, 'best_train_loss': 0.10781560898565594, 'max_val_acc': 0.9565058350563049, 'test_accuracy': 0.0, 'training_params': {'learning_rate': 0.0003, 'batch_size': 32, 'num_epochs': 50}}
Resultados guardados correctamente en results/efficient_net.json
Experimento terminado para efficientnet_b5.sw_in12k__bn_drp_unfreeze_416


### *New Color_Jitter*:

#### Changed color_jitter to : "color_jitter": A.ColorJitter(brightness = 0.4, contrast = 0.4, saturation = 0.3, hue = 0.2, p = 0.7)
#### from "color_jitter": A.ColorJitter(brightness = 0.2, contrast = 0.2, saturation = 0.2, hue = 0.1, p = 0.5),
#### No center crop, added random_rotate in 15 degrees

In [2]:
run_experiment(
    selected_transforms = ["color_jitter", "horizontal_flip", "brightness_contrast", "noise", "blur", "random_rotate"],
    model_name = "efficientnet_b5.sw_in12k",
    base_name = "",
    extra_info = "bn_drp_unfreeze_416_cj_cp", 
    model_type = "efficientnet_b5.sw_in12k",
    num_classes = 47,
    training_params = {"learning_rate": 0.0003, "batch_size": 32, "num_epochs": 50},
    frozen = True,
    result_file = "results/efficient_net.json",
    normalize = False,
    use_float = True,
    resize_size = 416  # Aquí defines dinámicamente el tamaño de `resize`
)

Running experiment: efficientnet_b5.sw_in12k__bn_drp_unfreeze_416_cj_cp
Using device: cuda


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: joseflores1 (benjaminrromo-universidad-adolfo-ib-ez). Use `wandb login --relogin` to force relogin


  0%|          | 0/50 [00:00<?, ?it/s]

Epoch: 1/50 - Time: 251.81s - Train Loss: 1.4832, Train Accuracy: 0.7318 - Validation Loss: 0.4339, Validation Accuracy: 0.9211
Model checkpoint saved at epoch 1


  4%|▍         | 2/50 [06:51<2:38:12, 197.77s/it]

...

Epoch: 14/50 - Time: 163.00s - Train Loss: 0.0884, Train Accuracy: 0.9737 - Validation Loss: 0.2058, Validation Accuracy: 0.9477


 28%|██▊       | 14/50 [42:00<1:48:01, 180.04s/it]

Epoch: 15/50 - Time: 163.81s - Train Loss: 0.0835, Train Accuracy: 0.9749 - Validation Loss: 0.2046, Validation Accuracy: 0.9492
Early stopping triggered at epoch 9. Best Validation Loss: 0.1557


elapsed_time,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
train_acc,▁▆▇▇▇▇█████████
train_loss,█▃▂▂▂▁▁▁▁▁▁▁▁▁▁
val_acc,▁▄▆▆█▇█▇█▆▇▆▇▆▇
val_loss,█▃▂▂▁▂▁▂▁▂▂▂▂▂▂
elapsed_time,163.80784
epoch,15
train_acc,0.97485
train_loss,0.08345
val_acc,0.9492


Results to save: {'architecture': 'efficientnet_b5.sw_in12k__bn_drp_unfreeze_416_cj_cp', 'model_type': 'efficientnet_b5.sw_in12k', 'best_epoch': 9, 'best_val_loss': 0.15565675573774104, 'best_train_loss': 0.10694622329144171, 'max_val_acc': 0.9557748436927795, 'test_accuracy': 0.0, 'training_params': {'learning_rate': 0.0003, 'batch_size': 32, 'num_epochs': 50}}
Resultados guardados correctamente en results/efficient_net.json
Experimento terminado para efficientnet_b5.sw_in12k__bn_drp_unfreeze_416_cj_cp


In [8]:
file_path = "results/efficient_net.json"
criteria = ["max_val_acc", "best_val_loss"]
 
for crit in criteria:
    print(f"El mejor modelo según {crit} es {find_best_model(file_path, crit)["architecture"]} con {crit} de {find_best_model(file_path, crit)[f"{crit}"]}")

El mejor modelo según max_val_acc es efficientnet_b5.sw_in12k__bn_drp_unfreeze_416 con max_val_acc de 0.9565058350563049
El mejor modelo según best_val_loss es efficientnet_b5.sw_in12k__bn_drp_unfreeze_416_cj_cp con best_val_loss de 0.15565675573774104
